In [ ]:
import random
import pandas as pd
import numpy as np
import os, shutil
from tqdm import tqdm
from sklearn.utils import shuffle

from PIL import Image, ImageEnhance

In [ ]:
# 提取 3 组数据

os.mkdir('./Data')
os.mkdir('./Data/V1')
os.mkdir('./Data/V2')
os.mkdir('./Data/V3')
os.mkdir('./Data/V1/Train')
os.mkdir('./Data/V1/Val')
os.mkdir('./Data/V2/Train')
os.mkdir('./Data/V2/Val')
os.mkdir('./Data/V3/Train')
os.mkdir('./Data/V3/Val')
os.mkdir('./Data/Test')

for f_name in os.listdir('../input/step1tostep2/Predict/Train'):
    shutil.copy('../input/step1tostep2/Predict/Train/' + f_name, './Data/V1/Train')
for f_name in os.listdir('../input/step1tostep2/Predict/Validation'):
    shutil.copy('../input/step1tostep2/Predict/Validation/' + f_name, './Data/V1/Val')
    
for f_name in os.listdir('../input/step2tostep3/Train'):
    shutil.copy('../input/step2tostep3/Train/' + f_name, './Data/V2/Train')
for f_name in os.listdir('../input/step2tostep3/Validation'):
    shutil.copy('../input/step2tostep3/Validation/' + f_name, './Data/V2/Val')

    
def Nos(src_dir, tar_dir):
    for f_name in os.listdir(src_dir):
        img = Image.open(src_dir + '/' + f_name)
        
        img = ImageEnhance.Color(img).enhance(1.5)
        img = ImageEnhance.Brightness(img).enhance(1.5)
        img = ImageEnhance.Sharpness(img).enhance(1.5)
        img = ImageEnhance.Contrast(img).enhance(1.5)
        
        img.save(tar_dir + '/' + f_name)
    
Nos('./Data/V1/Train','./Data/V3/Train')
Nos('./Data/V1/Val','./Data/V3/Val')

for f_name in os.listdir('../input/step1tostep2/Train/B'):
    shutil.copy('../input/step1tostep2/Train/B/' + f_name, './Data/Test')

In [ ]:
[len(os.listdir('./Data/V1/Train')),
len(os.listdir('./Data/V1/Val')),
len(os.listdir('./Data/V2/Train')),
len(os.listdir('./Data/V2/Val')),
len(os.listdir('./Data/V3/Train')),
len(os.listdir('./Data/V3/Val')),
len(os.listdir('./Data/Test'))]

In [ ]:
path = '../input/plant-pathology-2020-fgvc7/train.csv'
df_train_all = pd.read_csv(path)

In [ ]:
def get_class(row):
    
    if row['multiple_diseases'] == 1:
        return 'multiple_diseases'
    
    elif row['rust'] == 1:
        return 'rust'
    
    elif row['scab'] == 1:
        return 'scab'
    
    else:
        return 'healthy'
    
df_train_all['target'] = df_train_all.apply(get_class, axis=1)

df_train_all.head()

In [ ]:
df_train_all['target'].value_counts()

In [ ]:
# 提取表格

df_train_all_shuffle = shuffle(df_train_all, random_state=101)

trainlist = []
vallist = []
testlist = []
for f_name in os.listdir('../input/step1tostep2/Predict/Train'):
    trainlist.append('Train_' + f_name[:-4])
for f_name in os.listdir('../input/step1tostep2/Predict/Validation'):
    vallist.append('Train_' + f_name[:-4])
for f_name in os.listdir('../input/step1tostep2/Train/B'):
    testlist.append('Train_' + f_name[:-4])
    
df_train = df_train_all_shuffle.loc[df_train_all_shuffle['image_id'].isin(trainlist),:]
df_val = df_train_all_shuffle.loc[df_train_all_shuffle['image_id'].isin(vallist),:]
df_test = df_train_all_shuffle.loc[df_train_all_shuffle['image_id'].isin(testlist),:]

In [ ]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

In [ ]:
# 基本表格 没有平衡

df_train.to_csv('./Data/df_train.csv.gz', compression='gzip', index=False)
df_val.to_csv('./Data/df_val.csv.gz', compression='gzip', index=False)
df_test.to_csv('./Data/df_test.csv.gz', compression='gzip', index=False)

In [ ]:
import shutil
shutil.make_archive("./Data", 'zip', "./Data")